In [6]:
!pip install --upgrade pip --quiet
!pip install --upgrade google-api-python-client --quiet
!pip install nltk --quiet
!pip install pandas --quiet
!pip install nrclex --quiet

In [7]:
import nltk
nltk.download('punkt')
# imports
import json
import pandas as pd

import googleapiclient
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.errors import HttpError

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nrclex import NRCLex
import datetime
from datetime import datetime
from nltk.tokenize import casual

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\me\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Step 3

In [8]:
right_comment_df = pd.read_csv('Project_yt_comments.csv')
right_title_df = pd.read_csv('Project_yt_titles.csv')
demo_df = pd.read_csv('combine_democ_comments.csv')

d:\junk\ipykernel_12244\1937675395.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  right_comment_df = pd.read_csv('Project_yt_comments.csv')


In [9]:
def textcleaner(row):
    row = str(row)
    row = row.lower()
    # remove punctuation
    row = re.sub(r'[^\w\s]', '', row)
    #remove urls
    row  = re.sub(r'http\S+', '', row)
    #remove mentions
    row = re.sub(r"(?<![@\w])@(\w{1,25})", '', row)
    #remove hashtags
    row = re.sub(r"(?<![#\w])#(\w{1,25})", '',row)
    #remove other special characters
    row = re.sub('[^A-Za-z .-]+', '', row)
        #remove digits
    row = re.sub('\d+', '', row)
    row = row.strip(" ")
    row = re.sub('\s+', ' ', row)
    return row
    
stopeng = set(stopwords.words('english'))
def remove_stop(text):
    try:
        words = text.split(' ')
        valid = [x for x in words if x not in stopeng]
        return(' '.join(valid))
    except AttributeError:
        return('')

In [10]:
# Drop NaN
right_comment_df = right_comment_df.dropna()
right_title_df = right_title_df.dropna()
demo_df = demo_df.dropna()

In [11]:
# Change from datetime to date
right_comment_df['CommentCreationTime'] = right_comment_df['CommentCreationTime'].apply(lambda x: datetime.strptime(str(x)[0:10], '%Y-%m-%d').date())
right_title_df['published_at'] = right_title_df['published_at'].apply(lambda x: datetime.strptime(str(x)[0:10], '%Y-%m-%d').date())
demo_df['CommentCreationTime'] = demo_df['CommentCreationTime'].apply(lambda x: datetime.strptime(str(x)[0:10], '%Y-%m-%d').date())
demo_df['published_at'] = demo_df['published_at'].apply(lambda x: datetime.strptime(str(x)[0:10], '%Y-%m-%d').date())

In [12]:
# Tokenize
right_comment_df['TweetToken'] = right_comment_df['CommentTitle'].apply(lambda x: casual.TweetTokenizer().tokenize(x))
right_title_df['TweetToken'] = right_title_df['title'].apply(lambda x: casual.TweetTokenizer().tokenize(x))
demo_df['TweetTokenTitle'] = demo_df['title'].apply(lambda x: casual.TweetTokenizer().tokenize(x))
demo_df['TweetTokenComment'] = demo_df['CommentTitle'].apply(lambda x: casual.TweetTokenizer().tokenize(x))


In [13]:
# Clean text
right_comment_df['CommentCleaned'] = right_comment_df['TweetToken'].apply(lambda x: remove_stop(textcleaner(x)))
right_title_df['TitleCleaned'] = right_title_df['TweetToken'].apply(lambda x: remove_stop(textcleaner(x)))
demo_df['TitleCleaned'] = demo_df['TweetTokenTitle'].apply(lambda x: remove_stop(textcleaner(x)))
demo_df['CommentCleaned'] = demo_df['TweetTokenComment'].apply(lambda x: remove_stop(textcleaner(x)))


In [14]:
def nrc_sen(text, cat):
    sen = NRCLex(text)
    if cat == 'pos':
        return sen.affect_frequencies['positive']
    else:
        return sen.affect_frequencies['negative']

In [15]:
right_comment_df['PositiveScore'] = right_comment_df['CommentCleaned'].apply(lambda x: nrc_sen(x, 'pos'))
right_comment_df['NegativeScore'] = right_comment_df['CommentCleaned'].apply(lambda x: nrc_sen(x, 'neg'))        
right_title_df['PositiveScore'] = right_title_df['TitleCleaned'].apply(lambda x: nrc_sen(x, 'pos'))
right_title_df['NegativeScore'] = right_title_df['TitleCleaned'].apply(lambda x: nrc_sen(x, 'neg'))        

demo_df['PositiveScoreTitle'] = demo_df['TitleCleaned'].apply(lambda x: nrc_sen(x, 'pos'))
demo_df['NegativeScoreTitle'] = demo_df['TitleCleaned'].apply(lambda x: nrc_sen(x, 'neg'))    
demo_df['PositiveScoreComment'] = demo_df['CommentCleaned'].apply(lambda x: nrc_sen(x, 'pos'))    
demo_df['NegativeScoreComment'] = demo_df['CommentCleaned'].apply(lambda x: nrc_sen(x, 'neg'))        

In [16]:
def nrc_emo(text, ver):
    emo = NRCLex(text).affect_frequencies
    max_emo = max(emo, key=emo.get)
    max_score = emo[max_emo]
    if ver == 'score':
        return max_score
    else:
        return max_emo

In [17]:
right_comment_df['Emotion'] = right_comment_df['CommentCleaned'].apply(lambda x: nrc_emo(x, 'emo'))
right_comment_df['EmotionScore'] = right_comment_df['CommentCleaned'].apply(lambda x: nrc_emo(x, 'score'))        
right_title_df['Emotion'] = right_title_df['TitleCleaned'].apply(lambda x: nrc_emo(x, 'emo'))
right_title_df['EmotionScore'] = right_title_df['TitleCleaned'].apply(lambda x: nrc_emo(x, 'score'))        

demo_df['EmotionTitle'] = demo_df['TitleCleaned'].apply(lambda x: nrc_emo(x, 'emo'))
demo_df['EmotionScoreTitle'] = demo_df['TitleCleaned'].apply(lambda x: nrc_emo(x, 'score'))    
demo_df['EmotionComment'] = demo_df['CommentCleaned'].apply(lambda x: nrc_emo(x, 'emo'))
demo_df['EmotionScoreComment'] = demo_df['CommentCleaned'].apply(lambda x: nrc_emo(x, 'score'))    

In [18]:
demo_df.head()

,channel,video_id,title,keyword,published_at,CommentId,CommentTitle,CommentCreationTime,CommentLikes,TweetTokenTitle,...,TitleCleaned,CommentCleaned,PositiveScoreTitle,NegativeScoreTitle,PositiveScoreComment,NegativeScoreComment,EmotionTitle,EmotionScoreTitle,EmotionComment,EmotionScoreComment
0,Vice,SwoRx3tstxY,We Uncovered an ISIS Mass Grave | Super Users,ISIS,2022-04-11,Ugws1dFQrp7AovnexrB4AaABAg,Bless the hard work of journalists! Seeing the...,2022-04-12,146,"[We, Uncovered, an, ISIS, Mass, Grave, |, Supe...",...,uncovered isis mass grave super users,bless hard work journalists seeing deplorable ...,0.0,0.333333,0.066667,0.133333,fear,0.333333,fear,0.200000
1,Vice,SwoRx3tstxY,We Uncovered an ISIS Mass Grave | Super Users,ISIS,2022-04-11,UgxcNLZW2rAeMBklWD14AaABAg,Also I can't imagine the amount mental trauma...,2022-04-11,726,"[We, Uncovered, an, ISIS, Mass, Grave, |, Supe...",...,uncovered isis mass grave super users,also cant imagine amount mental trauma work pu...,0.0,0.333333,0.230769,0.076923,fear,0.333333,positive,0.230769
2,Vice,SwoRx3tstxY,We Uncovered an ISIS Mass Grave | Super Users,ISIS,2022-04-11,UgzFcqbEHILJ93hjvqh4AaABAg,This is so heartbreaking. What a horrific disp...,2022-04-11,251,"[We, Uncovered, an, ISIS, Mass, Grave, |, Supe...",...,uncovered isis mass grave super users,heartbreaking horrific display violence sorry ...,0.0,0.333333,0.000000,0.222222,fear,0.333333,fear,0.222222
3,Vice,SwoRx3tstxY,We Uncovered an ISIS Mass Grave | Super Users,ISIS,2022-04-11,UgxYrGf2RJZeKbFEap14AaABAg,7:35 Social-media shouldn't be just summarily ...,2022-04-11,219,"[We, Uncovered, an, ISIS, Mass, Grave, |, Supe...",...,uncovered isis mass grave super users,socialmedia shouldnt summarily deleting flagge...,0.0,0.333333,0.166667,0.166667,fear,0.333333,fear,0.166667
4,Vice,SwoRx3tstxY,We Uncovered an ISIS Mass Grave | Super Users,ISIS,2022-04-11,Ugx5rmcME6ua2pMkojt4AaABAg,VICE NEVER Dissapoints! Amazing documentaries!...,2022-04-11,127,"[We, Uncovered, an, ISIS, Mass, Grave, |, Supe...",...,uncovered isis mass grave super users,vice never dissapoints amazing documentaries t...,0.0,0.333333,0.285714,0.071429,fear,0.333333,positive,0.285714
